A template of the RL training

In [33]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

from utils import print_log

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# load the created dataset
dataset_folder_path = Path("dataset", "20250707", "split")

In [36]:
# copied from 03_data_split.ipynb

# Helper functions for the new split folder structure
def load_split_data_from_folder(split_folder, split_type='train'):
    """Load aggregate data from split folder"""
    segments = []
    with open(split_folder / f'{split_type}_segments.txt', 'r') as f:
        for line in f:
            start_str, end_str = line.strip().split(' - ')
            start = datetime.fromisoformat(start_str)
            end = datetime.fromisoformat(end_str)
            segments.append((start, end))
    
    df = pd.read_pickle(split_folder / f'{split_type}_aggregate_df.pkl')
    return segments, df

def load_signatures_from_split_folder(split_folder, split_type, appliance):
    """Load appliance signatures from split folder"""
    sig_path = split_folder / 'load_signature_library' / split_type / appliance / 'load_signatures.pkl'
    ranges_path = split_folder / 'load_signature_library' / split_type / appliance / 'selected_ranges.txt'
    
    if not sig_path.exists():
        return pd.DataFrame(), []
    
    signatures_df = pd.read_pickle(sig_path)
    ranges = []
    if ranges_path.exists():
        with open(ranges_path, 'r') as f:
            for line in f:
                start, end = map(int, line.strip().split(','))
                ranges.append((start, end))
    
    return signatures_df, ranges

In [37]:
aggregate_load_segments_train, aggregate_load_df_train = load_split_data_from_folder(dataset_folder_path, 'train')
aggregate_load_segments_test, aggregate_load_df_test = load_split_data_from_folder(dataset_folder_path, 'test') 
aggregate_load_segments_validation, aggregate_load_df_validation = load_split_data_from_folder(dataset_folder_path, 'val')

In [38]:
aggregate_load_df_train

,timestamp,aggregate,datetime,washing_machine,dishwasher,fridge,kettle,microwave,toaster,tv,htpc,gas_oven,kitchen_lights
0,1.357603e+09,234.0,2013-01-08 00:00:05+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,69.0,NaN,0.0
1,1.357603e+09,231.0,2013-01-08 00:00:11+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
2,1.357603e+09,234.0,2013-01-08 00:00:17+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
3,1.357603e+09,232.0,2013-01-08 00:00:23+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,68.0,NaN,0.0
4,1.357603e+09,232.0,2013-01-08 00:00:30+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231636,1.388448e+09,178.0,2013-12-30 23:59:35+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231637,1.388448e+09,177.0,2013-12-30 23:59:41+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231638,1.388448e+09,178.0,2013-12-30 23:59:47+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231639,1.388448e+09,178.0,2013-12-30 23:59:53+00:00,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0


In [39]:
aggregate_load_segments_train

[(datetime.datetime(2013, 1, 8, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 1, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 2, 27, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 2, 28, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 3, 8, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 3, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 3, 28, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 3, 31, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 3, 22, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 3, 26, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 4, 8, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2013, 4, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)),
 (datetime.datetime(2013, 4, 26, 0, 0, tzinfo=datetime.timezone.utc),
  d

In [40]:
from rl_env.env_data_loader import SmartMeterDataLoader

sm_dl = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_train,
    aggregate_load_df=aggregate_load_df_train
)

sm_dl.get_divided_segments_length()

162

In [41]:
sm_dl.divided_segments[7]

array([datetime.datetime(2013, 3, 10, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2013, 3, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)],
      dtype=object)

In [42]:
load_segment = sm_dl.get_aggregate_load_segment(13)

load_segment

,timestamp,aggregate,datetime
104747,1.363997e+09,335.0,2013-03-23 00:00:05+00:00
104748,1.363997e+09,336.0,2013-03-23 00:00:11+00:00
104749,1.363997e+09,333.0,2013-03-23 00:00:17+00:00
104750,1.363997e+09,334.0,2013-03-23 00:00:24+00:00
104751,1.363997e+09,331.0,2013-03-23 00:00:30+00:00
...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30+00:00
118502,1.364083e+09,171.0,2013-03-23 23:59:37+00:00
118503,1.364083e+09,171.0,2013-03-23 23:59:43+00:00
118504,1.364083e+09,171.0,2013-03-23 23:59:49+00:00


In [43]:
load_segment['datetime'] = pd.to_datetime(load_segment['timestamp'], unit='s')

load_segment

,timestamp,aggregate,datetime
104747,1.363997e+09,335.0,2013-03-23 00:00:05
104748,1.363997e+09,336.0,2013-03-23 00:00:11
104749,1.363997e+09,333.0,2013-03-23 00:00:17
104750,1.363997e+09,334.0,2013-03-23 00:00:24
104751,1.363997e+09,331.0,2013-03-23 00:00:30
...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30
118502,1.364083e+09,171.0,2013-03-23 23:59:37
118503,1.364083e+09,171.0,2013-03-23 23:59:43
118504,1.364083e+09,171.0,2013-03-23 23:59:49


(Optional) Load the pre-trained H-network and related components

In final product, the H-network should be trained along with the DDQL/PPO agent

In [44]:
import torch
from model.H_network.h_network import HNetwork

h_network_path = Path("model_trained", "h_network_20250708.pth")

h_network = HNetwork(2, 44, 1)
h_network.load_state_dict(torch.load(h_network_path))
h_network.eval()

HNetwork(
  (LSTM_1): LSTM(2, 44, batch_first=True, bidirectional=True)
  (ac1): Tanh()
  (LSTM_2): LSTM(88, 1, batch_first=True, bidirectional=True)
  (ac2): Tanh()
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

In [45]:
h_network_stdscaler_path = Path("model_trained", "h_network_standardscaler_20250708.pkl")
import joblib
h_network_stdscaler = joblib.load(h_network_stdscaler_path)

Create the environment

In [46]:
import sys
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env import SmartMeterWorld

env = SmartMeterWorld(
    aggregate_load_df=load_segment,
    render_mode="human",
)

env.set_h_network(h_network)
env.set_h_network_stdscaler(h_network_stdscaler)

[2025-07-10 19:18:57:258] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=93, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 38414), raddr=('127.0.0.1', 50007)>


In [47]:
from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!


/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:334: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:434: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantiating the environment through `gymnasium.make`
  logger.warn(


In [48]:
obs = env.reset()
obs

({'aggregate_load': array([335.], dtype=float32),
  'battery_soc': array([0.05382102], dtype=float32),
  'timestamp_features': array([-0.5       ,  0.33333334, -0.25      ], dtype=float32)},
 {})

In [49]:
# initialize a PPO agent
from stable_baselines3 import PPO

rl_datetime = datetime.now()
tensorboard_log_path = Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}")

rl_model = PPO(
    "MultiInputPolicy", 
    env, 
    verbose=2,
    tensorboard_log=tensorboard_log_path
)

rl_model.learn(
    total_timesteps=300000,
    progress_bar=True,
    tb_log_name="PPO_SmartMeterWorld"
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to rl_model/PPO/20250710_191857/PPO_SmartMeterWorld_1


Output()

-----------------------------
| time/              |      |
|    fps             | 117  |
|    iterations      | 1    |
|    time_elapsed    | 17   |
|    total_timesteps | 2048 |
-----------------------------


-------------------------------------------
| time/                   |               |
|    fps                  | 115           |
|    iterations           | 2             |
|    time_elapsed         | 35            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00043522645 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.42         |
|    explained_variance   | -1.79e-06     |
|    learning_rate        | 0.0003        |
|    loss                 | 1.08e+08      |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000339     |
|    std                  | 1             |
|    value_loss           | 2.3e+08       |
-------------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 116         |
|    iterations           | 3           |
|    time_elapsed         | 52          |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.002173535 |
|    clip_fraction        | 0.000684    |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.42       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 1.34e+08    |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.00198    |
|    std                  | 1           |
|    value_loss           | 2.63e+08    |
-----------------------------------------


...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.38e+04      |
|    ep_rew_mean          | 1.83e+07      |
| time/                   |               |
|    fps                  | 122           |
|    iterations           | 143           |
|    time_elapsed         | 2395          |
|    total_timesteps      | 292864        |
| train/                  |               |
|    approx_kl            | 0.00024632586 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.43         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 2.51e+08      |
|    n_updates            | 1420          |
|    policy_gradient_loss | -0.000252     |
|    std                  | 1.01          |
|    value_loss           | 4.95e+08      |
-------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.38e+04    |
|    ep_rew_mean          | 1.83e+07    |
| time/                   |             |
|    fps                  | 122         |
|    iterations           | 144         |
|    time_elapsed         | 2410        |
|    total_timesteps      | 294912      |
| train/                  |             |
|    approx_kl            | 0.000298306 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.43       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 2.88e+08    |
|    n_updates            | 1430        |
|    policy_gradient_loss | -0.000362   |
|    std                  | 1.01        |
|    value_loss           | 5.71e+08    |
-----------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.38e+04      |
|    ep_rew_mean          | 1.83e+07      |
| time/                   |               |
|    fps                  | 122           |
|    iterations           | 145           |
|    time_elapsed         | 2425          |
|    total_timesteps      | 296960        |
| train/                  |               |
|    approx_kl            | 0.00051718083 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.43         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 3.03e+08      |
|    n_updates            | 1440          |
|    policy_gradient_loss | -0.000607     |
|    std                  | 1.01          |
|    value_loss           | 5.94e+08      |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.38e+04      |
|    ep_rew_mean          | 1.83e+07      |
| time/                   |               |
|    fps                  | 122           |
|    iterations           | 146           |
|    time_elapsed         | 2440          |
|    total_timesteps      | 299008        |
| train/                  |               |
|    approx_kl            | 1.9569707e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.43         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 2.86e+08      |
|    n_updates            | 1450          |
|    policy_gradient_loss | 1.52e-05      |
|    std                  | 1.01          |
|    value_loss           | 5.75e+08      |
-------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.38e+04     |
|    ep_rew_mean          | 1.83e+07     |
| time/                   |              |
|    fps                  | 122          |
|    iterations           | 147          |
|    time_elapsed         | 2456         |
|    total_timesteps      | 301056       |
| train/                  |              |
|    approx_kl            | 4.769268e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.43        |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.0003       |
|    loss                 | 2.94e+08     |
|    n_updates            | 1460         |
|    policy_gradient_loss | -5.47e-05    |
|    std                  | 1.01         |
|    value_loss           | 5.79e+08     |
------------------------------------------


In [50]:
env.reset_render_window()

In [51]:
obs, info = env.reset()
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        print_log(f"Step: {env.current_step}, Action: {action}, Reward: {reward}")
        env.render()

[2025-07-10 20:35:06:370] Step: 1, Action: [0.88528097], Reward: 2.9484610037982466
[2025-07-10 20:35:06:379] Step: 2, Action: [0.8852801], Reward: 1.158057089406848
[2025-07-10 20:35:06:387] Step: 3, Action: [0.88528275], Reward: 1.782771721322934
[2025-07-10 20:35:06:394] Step: 4, Action: [0.885282], Reward: 3.132552595565041
[2025-07-10 20:35:06:401] Step: 5, Action: [0.8852845], Reward: 4.3714891865448555
[2025-07-10 20:35:06:410] Step: 6, Action: [0.88528454], Reward: 5.460555739656687
[2025-07-10 20:35:06:415] Step: 7, Action: [0.88528806], Reward: 6.31189403058811
[2025-07-10 20:35:06:421] Step: 8, Action: [0.8852892], Reward: 7.000847525537491
[2025-07-10 20:35:06:427] Step: 9, Action: [0.88529027], Reward: 7.595435328352213
[2025-07-10 20:35:06:433] Step: 10, Action: [0.8852918], Reward: 8.111209466182364
[2025-07-10 20:35:06:439] Step: 11, Action: [0.8852915], Reward: 8.613227648519398
[2025-07-10 20:35:06:445] Step: 12, Action: [0.8852926], Reward: 9.097737593362929
[2025-07

In [52]:
env.aggregate_load_df

,timestamp,aggregate,datetime,grid_load,battery_soc
104747,1.363997e+09,335.0,2013-03-23 00:00:05,3876.123867,0.184338
104748,1.363997e+09,336.0,2013-03-23 00:00:11,3877.120291,0.185076
104749,1.363997e+09,333.0,2013-03-23 00:00:17,3874.13102,0.185813
104750,1.363997e+09,334.0,2013-03-23 00:00:24,3875.12792,0.186674
104751,1.363997e+09,331.0,2013-03-23 00:00:30,3872.137934,0.187412
...,...,...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30,3724.610189,1.0
118502,1.364083e+09,171.0,2013-03-23 23:59:37,3715.701815,1.0
118503,1.364083e+09,171.0,2013-03-23 23:59:43,3715.701815,1.0
118504,1.364083e+09,171.0,2013-03-23 23:59:49,3715.701815,1.0


In [53]:
# save the graph

env.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph.png"))
)

In [67]:
env.close()

[2025-07-11 01:37:30:557] [SmartMeterWorld] Environment closed.


In [54]:
# save the model
rl_model_path = Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "rl_model.zip")
rl_model.save(rl_model_path)

---

In [64]:
# load the model & environment


env = SmartMeterWorld(
    aggregate_load_df=load_segment,
    render_mode="human",
)

env.set_h_network(h_network)
env.set_h_network_stdscaler(h_network_stdscaler)

rl_model_loaded = PPO.load(rl_model_path, env=env)

[2025-07-10 21:43:13:528] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=97, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 43478), raddr=('127.0.0.1', 50007)>
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [65]:
obs, info = env.reset()
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        print_log(f"Step: {env.current_step}, Action: {action}, Reward: {reward}")
        env.render()

[2025-07-10 21:43:16:202] Step: 1, Action: [0.8852958], Reward: 2.9483444876393077
[2025-07-10 21:43:16:209] Step: 2, Action: [0.88529474], Reward: 1.157978982493043
[2025-07-10 21:43:16:220] Step: 3, Action: [0.88529783], Reward: 1.782662393298871
[2025-07-10 21:43:16:227] Step: 4, Action: [0.88529694], Reward: 3.1324137634190716
[2025-07-10 21:43:16:232] Step: 5, Action: [0.88529986], Reward: 4.37132996958228
[2025-07-10 21:43:16:244] Step: 6, Action: [0.8853], Reward: 5.460382360626499
[2025-07-10 21:43:16:252] Step: 7, Action: [0.8853042], Reward: 6.311706489446164
[2025-07-10 21:43:16:265] Step: 8, Action: [0.8853057], Reward: 7.000642389080326
[2025-07-10 21:43:16:275] Step: 9, Action: [0.885307], Reward: 7.595228475265224
[2025-07-10 21:43:16:281] Step: 10, Action: [0.8853092], Reward: 8.11098973825264
[2025-07-10 21:43:16:287] Step: 11, Action: [0.8853088], Reward: 8.612986463120897
[2025-07-10 21:43:16:293] Step: 12, Action: [0.88531053], Reward: 9.097480958396357
[2025-07-10 

In [66]:
env.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph2.png"))
)